# EPR Baseline Correction

This notebook demonstrates the baseline correction functions available in the `epyr.baseline` module.

We will show different types of baseline distortions commonly found in EPR spectra and how to correct them using various algorithms.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys

# Add epyr to path
sys.path.append('../../')

from epyr.baseline import (
    baseline_polynomial,
    baseline_constant_offset,
    baseline_mono_exponential,
    baseline_stretched_exponential
)
from epyr.lineshapes import gaussian

# Create magnetic field range
B = np.linspace(-50, 50, 1000)  # mT

print("EPyR Baseline Correction module loaded successfully")

## Creating Synthetic EPR Data with Baseline Distortions

First, we create clean EPR spectra and add various types of baseline distortions commonly seen in experiments.

In [ ]:
# Create clean EPR spectrum (multi-component)
clean_spectrum = (
    0.8 * gaussian(B, center=-10, width=5.0, derivative=1) +
    1.0 * gaussian(B, center=5, width=7.0, derivative=1) +
    0.6 * gaussian(B, center=25, width=4.0, derivative=1)
)

# Create different types of baseline distortions
np.random.seed(42)

# 1. Linear baseline drift
linear_drift = 0.03 * B + 0.5
spectrum_linear = clean_spectrum + linear_drift

# 2. Quadratic baseline distortion
quadratic_drift = 0.0008 * B**2 - 0.02 * B + 0.3
spectrum_quadratic = clean_spectrum + quadratic_drift

# 3. Exponential decay baseline
exponential_baseline = 2.0 * np.exp(-B/15) + 0.2
spectrum_exponential = clean_spectrum + exponential_baseline

# 4. Constant offset
constant_offset = 1.5
spectrum_offset = clean_spectrum + constant_offset

# 5. Complex baseline (polynomial + exponential)
complex_baseline = 0.001 * B**3 - 0.05 * B**2 + 0.8 * np.exp(-abs(B)/20) + 0.7
spectrum_complex = clean_spectrum + complex_baseline

# Plot original spectra with different baselines
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

spectra = [
    (clean_spectrum, "Clean Spectrum", "g-"),
    (spectrum_linear, "Linear Drift", "b-"),
    (spectrum_quadratic, "Quadratic Drift", "r-"),
    (spectrum_exponential, "Exponential Decay", "m-"),
    (spectrum_offset, "Constant Offset", "c-"),
    (spectrum_complex, "Complex Baseline", "orange")
]

for i, (spectrum, title, color) in enumerate(spectra):
    axes[i].plot(B, spectrum, color, linewidth=2)
    axes[i].set_xlabel('Magnetic Field (mT)')
    axes[i].set_ylabel('EPR Signal')
    axes[i].set_title(title)
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Polynomial Baseline Correction

The most common baseline correction method using polynomial fitting.

In [ ]:
# Correct linear baseline (order 1 polynomial)
corrected_linear, baseline_fit_linear = baseline_polynomial(
    spectrum_linear, 
    x_data=B, 
    poly_order=1
)

# Correct quadratic baseline (order 2 polynomial)
corrected_quadratic, baseline_fit_quadratic = baseline_polynomial(
    spectrum_quadratic, 
    x_data=B, 
    poly_order=2
)

# Plot polynomial corrections
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Linear correction
axes[0, 0].plot(B, spectrum_linear, 'b-', label='Original', alpha=0.7)
axes[0, 0].plot(B, baseline_fit_linear, 'r--', label='Fitted Baseline', linewidth=2)
axes[0, 0].set_xlabel('Magnetic Field (mT)')
axes[0, 0].set_ylabel('EPR Signal')
axes[0, 0].set_title('Linear Baseline Fitting')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(B, clean_spectrum, 'g-', label='True Clean', alpha=0.8)
axes[0, 1].plot(B, corrected_linear, 'b-', label='Corrected', linewidth=2)
axes[0, 1].set_xlabel('Magnetic Field (mT)')
axes[0, 1].set_ylabel('EPR Signal')
axes[0, 1].set_title('Linear Baseline Correction Result')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Quadratic correction
axes[1, 0].plot(B, spectrum_quadratic, 'r-', label='Original', alpha=0.7)
axes[1, 0].plot(B, baseline_fit_quadratic, 'm--', label='Fitted Baseline', linewidth=2)
axes[1, 0].set_xlabel('Magnetic Field (mT)')
axes[1, 0].set_ylabel('EPR Signal')
axes[1, 0].set_title('Quadratic Baseline Fitting')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(B, clean_spectrum, 'g-', label='True Clean', alpha=0.8)
axes[1, 1].plot(B, corrected_quadratic, 'r-', label='Corrected', linewidth=2)
axes[1, 1].set_xlabel('Magnetic Field (mT)')
axes[1, 1].set_ylabel('EPR Signal')
axes[1, 1].set_title('Quadratic Baseline Correction Result')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate correction quality
linear_rmse = np.sqrt(np.mean((corrected_linear - clean_spectrum)**2))
quadratic_rmse = np.sqrt(np.mean((corrected_quadratic - clean_spectrum)**2))

print(f"Linear correction RMSE: {linear_rmse:.4f}")
print(f"Quadratic correction RMSE: {quadratic_rmse:.4f}")

## Baseline Correction with Signal Exclusion

When correcting baselines, it's important to exclude signal regions to avoid distorting the peaks.

In [ ]:
# Define signal regions to exclude from baseline fitting
# These regions contain EPR peaks that should not influence baseline fitting
exclude_regions = [(-20, 0), (0, 15), (15, 35)]  # Regions with EPR signals

# Correct with and without signal exclusion
corrected_no_exclusion, baseline_no_exclusion = baseline_polynomial(
    spectrum_quadratic,
    x_data=B,
    poly_order=2
)

corrected_with_exclusion, baseline_with_exclusion = baseline_polynomial(
    spectrum_quadratic,
    x_data=B,
    poly_order=2,
    exclude_regions=exclude_regions
)

# Plot comparison
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Without exclusion
axes[0, 0].plot(B, spectrum_quadratic, 'b-', label='Original', alpha=0.7)
axes[0, 0].plot(B, baseline_no_exclusion, 'r--', label='Baseline (No Exclusion)', linewidth=2)
axes[0, 0].set_xlabel('Magnetic Field (mT)')
axes[0, 0].set_ylabel('EPR Signal')
axes[0, 0].set_title('Baseline Fitting Without Signal Exclusion')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(B, clean_spectrum, 'g-', label='True Clean', alpha=0.8)
axes[0, 1].plot(B, corrected_no_exclusion, 'b-', label='Corrected (No Exclusion)', linewidth=2)
axes[0, 1].set_xlabel('Magnetic Field (mT)')
axes[0, 1].set_ylabel('EPR Signal')
axes[0, 1].set_title('Correction Result Without Exclusion')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# With exclusion
axes[1, 0].plot(B, spectrum_quadratic, 'b-', label='Original', alpha=0.7)
axes[1, 0].plot(B, baseline_with_exclusion, 'orange', linestyle='--', label='Baseline (With Exclusion)', linewidth=2)
# Shade excluded regions
for start, end in exclude_regions:
    axes[1, 0].axvspan(start, end, alpha=0.2, color='red', label='Excluded' if start == exclude_regions[0][0] else "")
axes[1, 0].set_xlabel('Magnetic Field (mT)')
axes[1, 0].set_ylabel('EPR Signal')
axes[1, 0].set_title('Baseline Fitting With Signal Exclusion')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(B, clean_spectrum, 'g-', label='True Clean', alpha=0.8)
axes[1, 1].plot(B, corrected_with_exclusion, 'orange', label='Corrected (With Exclusion)', linewidth=2)
axes[1, 1].set_xlabel('Magnetic Field (mT)')
axes[1, 1].set_ylabel('EPR Signal')
axes[1, 1].set_title('Correction Result With Exclusion')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Compare correction quality
rmse_no_exclusion = np.sqrt(np.mean((corrected_no_exclusion - clean_spectrum)**2))
rmse_with_exclusion = np.sqrt(np.mean((corrected_with_exclusion - clean_spectrum)**2))

print(f"RMSE without signal exclusion: {rmse_no_exclusion:.4f}")
print(f"RMSE with signal exclusion: {rmse_with_exclusion:.4f}")
print(f"Improvement factor: {rmse_no_exclusion/rmse_with_exclusion:.2f}x")

## Constant Offset Correction

Simple baseline correction for constant vertical offsets.

In [ ]:
# Correct constant offset using different methods

# Method 1: Use mean of offset region (automatic)
corrected_offset_mean, baseline_offset_mean = baseline_constant_offset(
    spectrum_offset,
    method='mean'
)

# Method 2: Use median of offset region
corrected_offset_median, baseline_offset_median = baseline_constant_offset(
    spectrum_offset,
    method='median'
)

# Method 3: Use specific region for offset calculation
# Use the first and last 50 points (edges of spectrum)
offset_region = (0, 50)  # First 50 points
corrected_offset_region, baseline_offset_region = baseline_constant_offset(
    spectrum_offset,
    offset_region_indices=offset_region,
    method='mean'
)

# Plot constant offset corrections
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

axes[0, 0].plot(B, spectrum_offset, 'c-', label='Original (with offset)', alpha=0.7)
axes[0, 0].axhline(y=baseline_offset_mean[0], color='r', linestyle='--', 
                   label=f'Detected offset: {baseline_offset_mean[0]:.3f}', linewidth=2)
axes[0, 0].set_xlabel('Magnetic Field (mT)')
axes[0, 0].set_ylabel('EPR Signal')
axes[0, 0].set_title('Constant Offset Detection')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Compare correction methods
axes[0, 1].plot(B, clean_spectrum, 'g-', label='True Clean', alpha=0.8, linewidth=2)
axes[0, 1].plot(B, corrected_offset_mean, 'c-', label='Mean Method', alpha=0.7)
axes[0, 1].plot(B, corrected_offset_median, 'm--', label='Median Method', alpha=0.7)
axes[0, 1].set_xlabel('Magnetic Field (mT)')
axes[0, 1].set_ylabel('EPR Signal')
axes[0, 1].set_title('Constant Offset Correction Methods')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Before and after comparison
axes[1, 0].plot(B, spectrum_offset, 'c-', label='Before Correction', alpha=0.7)
axes[1, 0].plot(B, corrected_offset_mean, 'b-', label='After Correction', linewidth=2)
axes[1, 0].set_xlabel('Magnetic Field (mT)')
axes[1, 0].set_ylabel('EPR Signal')
axes[1, 0].set_title('Before vs After Offset Correction')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Show regional offset detection
axes[1, 1].plot(B, spectrum_offset, 'c-', label='Original', alpha=0.7)
axes[1, 1].plot(B[:offset_region[1]], spectrum_offset[:offset_region[1]], 'ro', 
                markersize=2, label='Offset Region', alpha=0.5)
axes[1, 1].plot(B, corrected_offset_region, 'navy', label='Corrected (Regional)', linewidth=2)
axes[1, 1].set_xlabel('Magnetic Field (mT)')
axes[1, 1].set_ylabel('EPR Signal')
axes[1, 1].set_title('Regional Offset Correction')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Detected offset (mean method): {baseline_offset_mean[0]:.4f}")
print(f"Detected offset (median method): {baseline_offset_median[0]:.4f}")
print(f"True offset: {constant_offset:.4f}")
print(f"Error (mean): {abs(baseline_offset_mean[0] - constant_offset):.4f}")

## Exponential Baseline Correction

For baselines with exponential decay, often seen in EPR due to instrumental effects.

In [ ]:
# Mono-exponential baseline correction
corrected_mono_exp, baseline_mono_exp, params_mono = baseline_mono_exponential(
    spectrum_exponential,
    x_data=B,
    exclude_regions=exclude_regions  # Exclude signal regions
)

# Stretched exponential baseline correction
corrected_stretched_exp, baseline_stretched_exp, params_stretched = baseline_stretched_exponential(
    spectrum_exponential,
    x_data=B,
    exclude_regions=exclude_regions  # Exclude signal regions
)

# Plot exponential corrections
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Original with fitted baselines
axes[0, 0].plot(B, spectrum_exponential, 'b-', label='Original', alpha=0.7)
axes[0, 0].plot(B, exponential_baseline, 'g--', label='True Baseline', linewidth=2, alpha=0.8)
axes[0, 0].plot(B, baseline_mono_exp, 'r--', label='Mono-exp Fit', linewidth=2)
axes[0, 0].plot(B, baseline_stretched_exp, 'm--', label='Stretched-exp Fit', linewidth=2)
# Show excluded regions
for start, end in exclude_regions:
    axes[0, 0].axvspan(start, end, alpha=0.2, color='gray')
axes[0, 0].set_xlabel('Magnetic Field (mT)')
axes[0, 0].set_ylabel('EPR Signal')
axes[0, 0].set_title('Exponential Baseline Fitting')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Correction results
axes[0, 1].plot(B, clean_spectrum, 'g-', label='True Clean', alpha=0.8, linewidth=2)
axes[0, 1].plot(B, corrected_mono_exp, 'r-', label='Mono-exp Corrected', alpha=0.7)
axes[0, 1].plot(B, corrected_stretched_exp, 'm-', label='Stretched-exp Corrected', alpha=0.7)
axes[0, 1].set_xlabel('Magnetic Field (mT)')
axes[0, 1].set_ylabel('EPR Signal')
axes[0, 1].set_title('Exponential Correction Results')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Before/after mono-exponential
axes[1, 0].plot(B, spectrum_exponential, 'b-', label='Before', alpha=0.7)
axes[1, 0].plot(B, corrected_mono_exp, 'r-', label='After (Mono-exp)', linewidth=2)
axes[1, 0].set_xlabel('Magnetic Field (mT)')
axes[1, 0].set_ylabel('EPR Signal')
axes[1, 0].set_title('Mono-exponential Correction')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Before/after stretched exponential
axes[1, 1].plot(B, spectrum_exponential, 'b-', label='Before', alpha=0.7)
axes[1, 1].plot(B, corrected_stretched_exp, 'm-', label='After (Stretched-exp)', linewidth=2)
axes[1, 1].set_xlabel('Magnetic Field (mT)')
axes[1, 1].set_ylabel('EPR Signal')
axes[1, 1].set_title('Stretched Exponential Correction')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Display fitted parameters
print("Mono-exponential fit parameters:")
if params_mono:
    for key, value in params_mono.items():
        print(f"  {key}: {value:.4f}")

print("\nStretched exponential fit parameters:")
if params_stretched:
    for key, value in params_stretched.items():
        print(f"  {key}: {value:.4f}")

# Calculate correction quality
mono_rmse = np.sqrt(np.mean((corrected_mono_exp - clean_spectrum)**2))
stretched_rmse = np.sqrt(np.mean((corrected_stretched_exp - clean_spectrum)**2))
print(f"\nCorrection quality:")
print(f"Mono-exponential RMSE: {mono_rmse:.4f}")
print(f"Stretched exponential RMSE: {stretched_rmse:.4f}")

## Comparison of Correction Methods

Compare all baseline correction methods on the same complex baseline distortion.

In [ ]:
# Apply different correction methods to complex baseline
methods = {
    'Linear (Order 1)': baseline_polynomial(spectrum_complex, x_data=B, poly_order=1, exclude_regions=exclude_regions)[0],
    'Quadratic (Order 2)': baseline_polynomial(spectrum_complex, x_data=B, poly_order=2, exclude_regions=exclude_regions)[0],
    'Cubic (Order 3)': baseline_polynomial(spectrum_complex, x_data=B, poly_order=3, exclude_regions=exclude_regions)[0],
    'Mono-exponential': baseline_mono_exponential(spectrum_complex, x_data=B, exclude_regions=exclude_regions)[0],
    'Stretched-exponential': baseline_stretched_exponential(spectrum_complex, x_data=B, exclude_regions=exclude_regions)[0]
}

# Plot all methods
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

# Original spectrum
axes[0].plot(B, spectrum_complex, 'b-', label='Distorted', alpha=0.7, linewidth=2)
axes[0].plot(B, clean_spectrum, 'g-', label='True Clean', alpha=0.8, linewidth=2)
axes[0].set_xlabel('Magnetic Field (mT)')
axes[0].set_ylabel('EPR Signal')
axes[0].set_title('Original Complex Baseline')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Individual method results
colors = ['red', 'purple', 'orange', 'brown', 'pink']
rmse_values = {}

for i, (method_name, corrected) in enumerate(methods.items()):
    axes[i+1].plot(B, clean_spectrum, 'g-', label='True Clean', alpha=0.8, linewidth=2)
    axes[i+1].plot(B, corrected, colors[i], label=f'{method_name}', linewidth=2, alpha=0.8)
    axes[i+1].set_xlabel('Magnetic Field (mT)')
    axes[i+1].set_ylabel('EPR Signal')
    axes[i+1].set_title(f'{method_name} Correction')
    axes[i+1].legend()
    axes[i+1].grid(True, alpha=0.3)
    
    # Calculate RMSE
    rmse = np.sqrt(np.mean((corrected - clean_spectrum)**2))
    rmse_values[method_name] = rmse

plt.tight_layout()
plt.show()

# Performance comparison
print("\nBaseline Correction Method Comparison (RMSE):")
print("=" * 45)
sorted_methods = sorted(rmse_values.items(), key=lambda x: x[1])
for method, rmse in sorted_methods:
    print(f"{method:<25}: {rmse:.4f}")

print(f"\nBest method: {sorted_methods[0][0]} (RMSE: {sorted_methods[0][1]:.4f})")

## Advanced: Custom Baseline Correction

Demonstrate how to chain multiple correction methods for complex baselines.

In [ ]:
# Create a very complex baseline with multiple components
very_complex_baseline = (
    0.002 * B**3 +                          # Cubic drift
    1.5 * np.exp(-B/20) +                   # Exponential decay  
    0.5 * np.exp(-(B+30)**2/200) +          # Gaussian bump
    2.0                                     # Constant offset
)
spectrum_very_complex = clean_spectrum + very_complex_baseline

# Multi-step correction approach
print("Multi-step baseline correction:")
print("Step 1: Remove exponential component")

# Step 1: Remove exponential decay
step1_corrected, step1_baseline, _ = baseline_stretched_exponential(
    spectrum_very_complex,
    x_data=B,
    exclude_regions=exclude_regions
)

print("Step 2: Remove polynomial drift from residual")
# Step 2: Remove remaining polynomial drift
step2_corrected, step2_baseline = baseline_polynomial(
    step1_corrected,
    x_data=B,
    poly_order=3,
    exclude_regions=exclude_regions
)

print("Step 3: Final offset adjustment")
# Step 3: Final offset correction
final_corrected, final_baseline = baseline_constant_offset(
    step2_corrected,
    method='median'
)

# Plot multi-step correction
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Original very complex baseline
axes[0, 0].plot(B, spectrum_very_complex, 'b-', label='Original', alpha=0.7)
axes[0, 0].plot(B, very_complex_baseline, 'r--', label='True Baseline', linewidth=2)
axes[0, 0].set_xlabel('Magnetic Field (mT)')
axes[0, 0].set_ylabel('EPR Signal')
axes[0, 0].set_title('Very Complex Baseline')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Step-by-step correction
axes[0, 1].plot(B, spectrum_very_complex, 'b-', label='Original', alpha=0.5)
axes[0, 1].plot(B, step1_corrected, 'orange', label='Step 1: Exp removed', alpha=0.7)
axes[0, 1].plot(B, step2_corrected, 'purple', label='Step 2: Poly removed', alpha=0.7)
axes[0, 1].plot(B, final_corrected, 'red', label='Step 3: Final', linewidth=2)
axes[0, 1].set_xlabel('Magnetic Field (mT)')
axes[0, 1].set_ylabel('EPR Signal')
axes[0, 1].set_title('Multi-step Correction Process')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Final result comparison
axes[1, 0].plot(B, clean_spectrum, 'g-', label='True Clean', alpha=0.8, linewidth=3)
axes[1, 0].plot(B, final_corrected, 'r-', label='Multi-step Corrected', linewidth=2, alpha=0.8)
axes[1, 0].set_xlabel('Magnetic Field (mT)')
axes[1, 0].set_ylabel('EPR Signal')
axes[1, 0].set_title('Final Correction Result')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Residuals
residuals = final_corrected - clean_spectrum
axes[1, 1].plot(B, residuals, 'k-', linewidth=1)
axes[1, 1].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[1, 1].set_xlabel('Magnetic Field (mT)')
axes[1, 1].set_ylabel('Residuals')
axes[1, 1].set_title('Correction Residuals')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate final correction quality
multi_step_rmse = np.sqrt(np.mean((final_corrected - clean_spectrum)**2))
single_step_cubic = baseline_polynomial(spectrum_very_complex, x_data=B, poly_order=3, exclude_regions=exclude_regions)[0]
single_step_rmse = np.sqrt(np.mean((single_step_cubic - clean_spectrum)**2))

print(f"\nCorrection Quality Comparison:")
print(f"Single-step cubic correction RMSE: {single_step_rmse:.4f}")
print(f"Multi-step correction RMSE: {multi_step_rmse:.4f}")
print(f"Improvement factor: {single_step_rmse/multi_step_rmse:.2f}x")

## Summary

The EPyR baseline correction module provides:

**Core Functions:**
- `baseline_polynomial(y_data, x_data, poly_order, exclude_regions)` - Polynomial fitting
- `baseline_constant_offset(y_data, method)` - Simple offset correction
- `baseline_mono_exponential(y_data, x_data, exclude_regions)` - Exponential decay fitting
- `baseline_stretched_exponential(y_data, x_data, exclude_regions)` - Advanced exponential fitting

**Key Features:**
- Signal region exclusion to preserve peak shapes
- Multiple polynomial orders (linear, quadratic, cubic, etc.)
- Exponential models for instrument-related baseline distortions
- Parameter extraction for fitted models
- Flexible region-of-interest specification

**Best Practices:**
- Always exclude signal regions from baseline fitting
- Start with simple methods (polynomial) before advanced ones
- Use multi-step correction for complex baselines
- Choose polynomial order based on baseline complexity
- Validate correction quality using residuals analysis

**Method Selection Guide:**
- **Linear drift**: `baseline_polynomial(poly_order=1)`
- **Curved baselines**: `baseline_polynomial(poly_order=2 or 3)`
- **Instrument decay**: `baseline_mono_exponential()` or `baseline_stretched_exponential()`
- **Simple offset**: `baseline_constant_offset()`
- **Complex baselines**: Multi-step approach combining methods